In [1]:
import pathlib

import numpy as np
import pandas as pd
import wisc_ecephys_tools as wet

In [2]:
condition_display_names = {
    "early_bsl_nrem": "Early.BSL.NREM",
    "early_rec_nrem_match": "Late.BSL.NREM",
    "early_ext_wake": "Early.EXT.Wake",
    "late_ext_wake": "Late.EXT.Wake",
    "early_rec_nrem": "Early.REC.NREM",
    "late_rec_nrem": "Late.REC.NREM",
}

measure_display_names = {
    "firing_rate": "Firing.Rate",
    "on_firing_rate": "ON.Firing.Rate",
}


def use_display_names(df: pd.DataFrame, minimize: bool = True) -> pd.DataFrame:
    essential_cols = [
        "subject",
        "probe",
        "cluster_id",
        "region",
        "petersen_cell_type",
        "condition",
    ]
    assert all(col in df.columns for col in essential_cols), (
        f"Missing one or more essential columns: {essential_cols}"
    )

    if minimize:
        df = df[df["condition"].isin(condition_display_names)]
        df = df[essential_cols + list(measure_display_names)]

    df["condition"] = df["condition"].map(condition_display_names)
    df.rename(columns=measure_display_names, inplace=True)

    return df.reset_index(drop=True)

In [ ]:
nbsh = wet.get_sglx_project("seahorse")
df = pd.read_parquet(nbsh.get_project_file("estm_on_frs.with_meta.sua_moderate.pqt"))
df = use_display_names(df)
df = df[df["Firing.Rate"] > 0]
df["Log.Firing.Rate"] = np.log10(df["Firing.Rate"])
df["Log.ON.Firing.Rate"] = np.log10(df["ON.Firing.Rate"])
df["Sqrt.Firing.Rate"] = np.sqrt(df["Firing.Rate"])
df["Sqrt.ON.Firing.Rate"] = np.sqrt(df["ON.Firing.Rate"])

savepath = pathlib.Path("../../r/f25a/inst/extdata/firing_rates.pqt")
assert savepath.parent.exists()
df.to_parquet(savepath)